# This is a deep leaning model which can detect emotions

In [37]:
# importing lab which we will use
import numpy as np
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os


In [38]:
num_classes = 7 # this is the no no class like happy suprise angry etc
img_rows,img_cols = 48,48 # this is the dim of row and columns
batch_size = 32 # this is the batch size


In [39]:
train_dir = 'C:\\Users\\Moosa\\Desktop\\cnn\\train' # this is the train folder of my computer plz use your own file path
validation_dir = 'C:\\Users\\Moosa\\Desktop\\cnn\\validation'# same hear use your own file path where you have stored the data

In [40]:
# we are genrating new images 
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=30,
                                   shear_range=0.3,zoom_range=0.3,
                                   width_shift_range=0.4,
                                   height_shift_range=0.4,
                                   horizontal_flip=True,
                                   fill_mode='nearest')


In [41]:
# we are resaling the images in val dataset
validation_datagen = ImageDataGenerator(rescale=1./255)

In [42]:
# genrating images
train_generator = train_datagen.flow_from_directory(train_dir,color_mode='grayscale'
                                                    ,target_size=(img_rows,img_cols)
                                                    ,batch_size=batch_size
                                                    ,class_mode='categorical'
                                                    ,shuffle=True)

Found 28789 images belonging to 7 classes.


In [43]:
# usign validation rescaling
validation_generator = validation_datagen.flow_from_directory(validation_dir,color_mode='grayscale'
                                                              ,target_size=(img_rows,img_cols),
                                                              batch_size=batch_size,
                                                              class_mode='categorical',
                                                              shuffle=True)

Found 3589 images belonging to 7 classes.


In [44]:
# making  a model
model = Sequential()

In [45]:
#first block
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [46]:
# Block-2 

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [47]:
# Block 3
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [48]:
# Block-4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [49]:
# Block 4
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block 5

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


In [50]:
# last Block
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))
# printing model summery
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
activation_23 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_24 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 24, 24, 32)       

In [51]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('emotion_predict.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1
                                )
earlystop = EarlyStopping(monitor='accuracy',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)



In [52]:

# compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

In [53]:
nb_train_samples = 24176
nb_validation_samples = 3006
epochs=25 # no. of epoch = 25 

In [54]:
# fitting the model
history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)


Epoch 1/25
755/755 [==============================] - 2217s 3s/step - loss: 2.2116 - accuracy: 0.2009 - val_loss: 1.8083 - val_accuracy: 0.2470
Epoch 2/25
755/755 [==============================] - 1911s 3s/step - loss: 1.8457 - accuracy: 0.2356 - val_loss: 1.7451 - val_accuracy: 0.2594
Epoch 3/25
755/755 [==============================] - 1803s 2s/step - loss: 1.8002 - accuracy: 0.2456 - val_loss: 1.7966 - val_accuracy: 0.2526
Epoch 4/25
755/755 [==============================] - 1881s 2s/step - loss: 1.7808 - accuracy: 0.2627 - val_loss: 1.6126 - val_accuracy: 0.2896
Epoch 5/25
755/755 [==============================] - 1839s 2s/step - loss: 1.7442 - accuracy: 0.2868 - val_loss: 1.7691 - val_accuracy: 0.3106
Epoch 6/25
755/755 [==============================] - 1911s 3s/step - loss: 1.7006 - accuracy: 0.3056 - val_loss: 1.6996 - val_accuracy: 0.3572
Epoch 7/25
755/755 [==============================] - 1801s 2s/step - loss: 1.6343 - accuracy: 0.3473 - val_loss: 1.4429 - val_accuracy:

In [64]:
model.save("mooosa.h5")

NameError: name 'loaded_model' is not defined